In [3]:
preparser(False)

In [12]:
R = PolynomialRing(QQ, ['x1', 'x2'])
x1, x2 = R.gens()

A = matrix(QQ, [[0, -1], [1, 0]])  # 90° rotation matrix
t = vector(QQ, [1, 1])            # translation vector

def S2(v): return A * v
def w2(v): return - v + t

v = vector([x1, x2])



def S2w2(v): return S2(w2(v))

S2w2_2 = S2w2(S2w2(v))

print("S2^4(v) =", S2(S2(S2(S2(v)))))
print("w2^2(v) =", w2(w2(v)))
print("(w2*S2)^4(v) =", w2(S2(w2(S2(w2(S2(w2(S2(v)))))))))
print("w2*S2^2", w2(S2(S2(v))))

S2^4(v) = (x1, x2)
w2^2(v) = (x1, x2)
(w2*S2)^4(v) = (x1, x2)
w2*S2^2 (x1 + 1, x2 + 1)


In [13]:
def B0_as_finitely_presented_group(N):
    """
    Construct the subgroup B_0 of the automorphism group of X_0(N) as finitely presented group. 
    Returns (B_0, invariants) and invariants is a dictionary containing v2, v3, mu, omega, v.
    
    Warning!!! At the moment this funtion returns some relations that are not proven to hold if v2>=5.
    Verify that the returned relations are satisfied before using this function in a proof.
    """
    N = Integer(N)
    if N <= 0:
        raise ValueError(f"N={N} must be a positive integer.")

    fac = N.factor()   # list of (prime, exponent)
    primes = N.prime_divisors()

    v2 = Integer(N).valuation(2)
    v3 = Integer(N).valuation(3)

    mu = min(3, v2 // 2)       
    omega = min(1, v3 // 2)
    v_val = 2**mu * 3**omega

    names = []
    # Build generator list: S2, S3, and w_p for each prime dividing N
    if 2 in primes:
        names = ["w2", "S2"]
    if 3 in primes:
        names += ["w3", "S3"]
    names += ["w{}".format(p) for p in primes if p > 3]
    F = FreeGroup(len(names),names)
    gens = list(F.gens())
    gens_map = F.gens_dict()

    if 2 in primes:
        S2 = gens_map["S2"]
    if 3 in primes:
        S3 = gens_map["S3"]
    

    rels = []

    # --- all w_p of order 2 and commute pairwise ---
    for p in primes:
        wp = gens_map[f"w{p}"]
        rels.append(wp**2)
    for i in range(len(primes)):
        for j in range(i+1, len(primes)):
            p, q = primes[i], primes[j]
            wp, wq = gens_map[f"w{p}"], gens_map[f"w{q}"]
            rels.append(wp * wq * (wq * wp)**(-1))

    # (S2)^{2^mu} = 1, (S3)^{3^omega} = 1
    if 2 in primes:
        rels.append(S2 ** (2**mu))
    if 3 in primes:
        rels.append(S3 ** (3**omega))

    # --- 3-part relations ---
    if v3 == 2:
        rels.append((gens_map["w3"] * S3) ** 3)
    if v3 >= 3:
        a = gens_map["w3"] * S3 * gens_map["w3"]
        rels.append(a * S3 * (S3 * a)**(-1))

    # --- 2-part relations ---
    if "w2" in gens_map:
        w2 = gens_map["w2"]
        if v2 in (2, 4, 6):
            # second relation is unproven at the moment if v2=6
            rels += [(w2 * S2)**3, w2 * S2**(-3) * w2 * S2**(-2) * w2 * S2 * w2 * S2**(-2)]
        elif v2 in (3, 5, 7):
            # second relation is unproven at the moment if v2=5,7
            # it can be replaced with the aslo unproven #(w2 * S2**(-1) * w2 * S2)**2 if v2=5
            rels += [(w2 * S2)**4, w2 * S2**3 * w2 * S2**2 * w2 * S2 * w2 * S2**(-2)]
        elif v2 == 8:
            rels.append((w2 * S2)**2 * (w2 * (S2**3))**2)
        elif v2 >= 9:
            b = w2 * S2 * w2
            rels.append(b * S2 * (S2 * b)**(-1))

    # S2 and S3 commute
    if 2 in primes and 3 in primes:
        rels.append(S2 * S3 * (S3 * S2)**(-1))

    # --- generic relations for w_p with S2, S3 ---
    for p in primes:
        wp = gens_map[f"w{p}"]
        pe = p**(N.valuation(p))
        if p != 2 and 2 in primes:
            rels.append(wp * S2 * wp * S2 ** (-pe%(2**mu)))
        if p != 3 and 3 in primes:
            rels.append(wp * S3 * wp * S3 ** (-pe%(3**omega)))

    # quotient
    B0 = F / rels

    invariants = {"v2": int(v2), "v3": int(v3), "mu": int(mu),
                  "omega": int(omega), "v": int(v_val), "primes": primes}

    return B0, invariants
    
    
def apply(m,g,ambient=False):
    g = list(g)
    M = m.parent()
    if ambient:
        M = M.ambient()
    gm = 0
    for a,c in m.modular_symbol_rep():
        gm += a*c.apply(g)
    return M(gm)

def atkin_lehner_divisors(N):
    al_divisors = []
    for d in divisors(N):
        if gcd(d, N // d) == 1:
            al_divisors.append(d)
    return al_divisors

def Sv_operator(S, v):
    assert 24%v == 0
    assert S.level()%(v**2)==0
    g = [v,1,0,v]
    rows = [S.coordinate_vector(apply(m,g)) for m in S.basis()]
    return Matrix(rows)

def cusp_permutation(G, m):
    C = G.cusps()
    if not isinstance(m,list):
        m = m.list()
    perm = [C.index(G.reduce_cusp(c.apply(m)))+1 for c in C]
    return Permutation(perm)

def group_relations(G, names):
    """Return the relations between the generators of the permuation group G
    the list given in names will be used as names for the variables in these relations.
    
    Warning!!!  If you did not create the permutation group G with canonicalize=False then
    the generators of G might not be what you expect them to be.
    """
    assert G.ngens() == len(names)
    F = FreeGroup(names)
    Gfp = G.as_finitely_presented_group()
    assert all([Gfp.gen(i)==Gfp(G.gen(i)) for i in range(G.ngens())])
    generator_mapping = {str(Gfp.gen(i)):F.gen(i) for i in range(G.ngens())}
    return [rel.subs(**generator_mapping) for rel in Gfp.relations()]
    
def E0_as_permutation_group(N, full_level=False):
    """Return the image of E_0 as acting on the cusps.
    A priori this could be a quotient of E_0. However in practice this 
    permutation representation is often faithfull.
    """
    N = ZZ(N)
    v2 = Integer(N).valuation(2)
    v3 = Integer(N).valuation(3)

    mu = min(3, v2 // 2)       
    omega = min(1, v3 // 2)
    
    if v2==0 and v3 == 0:
        return PermutationGroup([]),[]
    if full_level:
        G = Gamma0(N)
    else:
        G = Gamma0(2**v2*3**v3)

    gens = []
    names = []
    if v2 > 0: 
        w2 = cusp_permutation(G,G.atkin_lehner_matrix(2))
        S2 = cusp_permutation(G,[2**mu,1,0,2**mu])
        gens = [w2, S2]
        names = ["w2", "S2"]
    if v3 > 0:
        w3 = cusp_permutation(G,G.atkin_lehner_matrix(3))
        S3 = cusp_permutation(G,[3**omega,1,0,3**omega])
        gens += [w3, S3]
        names += ["w3", "S3"]
    return PermutationGroup(gens, canonicalize=False), names
    
def B0_as_permutation_group(N):
    N = ZZ(N)
    primes = [p for p in N.prime_divisors() if p > 3]
    AL_operators = tuple(cusp_permutation(G, G.atkin_lehner_matrix(p)) for p in primes)
    E0, names = E0_as_permutation_group(N, full_level=True)
    names += [f"w{p}" for p in primes]
    return PermutationGroup(E0.gens()+AL_operators, canonicalize=False), names
   

def E0_relations_on_cusps(N, full_level=False):
    E0, names = E0_as_permutation_group(N, full_level=full_level)
    return group_relations(E0, names)
    

def E0_as_matrix_group(N, weight=2, cuspidal=True):
    G = Gamma0(N)
    M = ModularSymbols(G, weight=weight)
    if cuspidal:
        S = M.cuspidal_submodule()
    else:
        S = M
        
    v2 = Integer(N).valuation(2)
    v3 = Integer(N).valuation(3)

    mu = min(3, v2 // 2)       
    omega = min(1, v3 // 2)
    gens = []
    names = []
    
    if v2 > 0: 
        w2 = S.atkin_lehner_operator(2).matrix()
        S2 = Sv_operator(S, 2**mu)
        gens = [w2, S2]
        names = ["w2", "S2"]
    if v3 > 0:
        w3 = S.atkin_lehner_operator(3).matrix()
        S3 = Sv_operator(S, 3**omega)
        gens += [w3, S3]
        names += ["w3", "S3"]
    return MatrixGroup(gens), names
    
def B0_as_matrix_group(N, weight=2, cuspidal=True):
    N = ZZ(N)
    G = Gamma0(N)
    M = ModularSymbols(G, weight=weight)
    
    if cuspidal:
        S = M.cuspidal_submodule()
    else:
        S = M
        

    primes = [p for p in N.prime_divisors() if p > 3]
    AL_operators = tuple(S.atkin_lehner_operator(p).matrix() for p in primes)
    E0, names = E0_as_matrix_group(N, weight=weight, cuspidal=cuspidal)
    names += [f"w{p}" for p in primes]
    return MatrixGroup(E0.gens()+AL_operators), names
   

In [14]:
E0, names = E0_as_permutation_group(2**8)
w2, S2 = E0.gens()
w2*S2**2*w2*S2==S2**2*w2*S2*w2

False

In [15]:
todo = [140, 180, 220, 252, 280, 288, 360, 440, 720 ]
for N in todo:
    B0, names =  B0_as_matrix_group(N)
    ZB0 = B0.center()
    print(N, B0.cardinality(), ZB0.cardinality(),ZB0.exponent())
    genera = [(z.matrix()-1).kernel().dimension()/2 for z in ZB0 if z.matrix()!=1]
    print(f"genera: {genera}")

140 24 4 2
genera: [10, 10, 7]


180 144 1 1
genera: []


220 24 4 2
genera: [16, 13, 10]


252 144 2 2
genera: [19]


280 32 8 2


...

288 384 2 2
genera: [17]


360 192 2 2
genera: [29]


440 32 8 2


genera: [33, 33, 25, 33, 33, 33, 33]


720 576 1 1
genera: []


In [16]:
todo =  [140, 180, 220, 252, 280, 288, 360, 440, 720 ]
for N in todo:
    B0, names =  B0_as_matrix_group(N)
    #ZB0 = B0.center()
    print(f"level: {N}, #B0({N})={B0.cardinality()}")
    genera = [(z.matrix()-1).kernel().dimension()/2 for z in B0 if z!=B0(1) and z**2==B0(1)]
    print(f"minimum  {min(genera)}, all genera: {genera}")

level: 140, #B0(140)=24


minimum  7, all genera: [10, 10, 7, 9, 8, 10, 7, 9, 8, 10, 7, 9, 8, 10, 7]


level: 180, #B0(180)=144


minimum  11, all genera: [11, 11, 11, 13, 11, 13, 11, 13, 11, 11, 11, 11, 11, 11, 11, 13, 11, 11, 11, 13, 11, 11, 11, 13, 11, 11, 11, 13, 11, 11, 11, 13, 11, 11, 11, 13, 11, 11, 11]


level: 220, #B0(220)=24


...

minimum  25, all genera: [25, 29, 29, 29, 29, 29, 29, 25, 29, 29, 25, 29, 29, 29, 29, 25, 29, 29, 29, 29, 25, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 25, 25, 25, 29, 25, 29, 25, 29, 29, 29, 25, 29, 29, 25, 29, 25, 29, 25, 29, 25, 25, 25, 29]


level: 440, #B0(440)=32


minimum  25, all genera: [33, 33, 25, 31, 33, 27, 29, 33, 31, 33, 27, 33, 31, 33, 27, 31, 33, 27, 29, 33, 33, 33, 33]


level: 720, #B0(720)=576


minimum  57, all genera: [61, 61, 57, 61, 57, 57, 61, 61, 61, 57, 57, 61, 57, 61, 57, 61, 57, 61, 57, 61, 57, 61, 57, 61, 57, 61, 57, 61, 57, 61, 57, 61, 57, 61, 57, 61, 57, 61, 57, 61, 57, 57, 57, 57, 61, 61, 57, 61, 61, 57, 61, 61, 61, 57, 57, 61, 57, 61, 57, 57, 57, 57, 57, 61, 57, 57, 61, 57, 57, 61, 57, 57, 61, 57, 57, 61, 57, 57, 61, 57, 57, 61, 57, 57, 61, 57, 57, 61, 61, 57, 57, 57, 57, 61, 61, 57, 57, 57, 57]


In [54]:
for i in range(1,12):
    B0,invariants = B0_as_finitely_presented_group(2**i)
    for N in range(1,20,2):
        if N%3==0:
            continue
        E0, names = E0_as_permutation_group(2**i*N, full_level=True)
        print(i, N, E0.cardinality(),E0_relations_on_cusps(2**i*N, full_level=True))
        
        phi = B0.hom(E0.gens(), E0)
        # sage doesn't have is_isomorphism for phi
        # so we do some cadrinality checks which are equivalent
        assert phi.kernel().cardinality() == 1
        assert B0.cardinality() == E0.cardinality()
        
        


1 1 2 [S2, w2^2]
1 5 2 [S2, w2^2]
1 7 2 [S2, w2^2]
1 11 2 [S2, w2^2]
1 13 2 [S2, w2^2]
1 17 2 [S2, w2^2]
1 19 2 [S2, w2^2]
2 1 6 [w2^2, S2^2, (S2*w2)^3]
2 5 6 [w2^2, S2^2, (S2*w2)^3]
2 7 6 [w2^2, S2^2, (w2*S2)^3]
2 11 6 [w2^2, S2^2, (w2*S2)^3]
2 13 6 [w2^2, S2^2, (S2*w2)^3]
2 17 6 [w2^2, S2^2, (w2*S2)^3]
2 19 6 [w2^2, S2^2, (S2*w2)^3]
3 1 8 [w2^2, S2^2, (S2*w2)^4]
3 5 8 [w2^2, S2^2, (S2*w2)^4]
3 7 8 [w2^2, S2^2, (S2*w2)^4]
3 11 8 [w2^2, S2^2, (S2*w2)^4]
3 13 8 [w2^2, S2^2, (S2*w2)^4]
3 17 8 [w2^2, S2^2, (S2*w2)^4]
3 19 8 [w2^2, S2^2, (S2*w2)^4]
4 1 24 [w2^2, S2^4, (w2*S2^-1)^3]
4 5 24 [w2^2, S2^4, (w2*S2^-1)^3]
4 7 24 [w2^2, S2^4, (w2*S2^-1)^3]
4 11 24 [w2^2, S2^4, (w2*S2^-1)^3]
4 13 24 [w2^2, S2^4, (w2*S2^-1)^3]
4 17 24 [w2^2, S2^4, (w2*S2^-1)^3]
4 19 24 [w2^2, S2^4, (w2*S2^-1)^3]
5 1 32 [w2^2, S2^4, (w2*S2^-1*w2*S2)^2, (w2*S2^-1)^4]
5 5 32 [w2^2, S2^4, (w2*S2^-1*w2*S2)^2, (w2*S2^-1)^4]
5 7 32 [w2^2, S2^4, (w2*S2^-1*w2*S2)^2, (w2*S2^-1)^4]
5 11 32 [w2^2, S2^4, (w2*S2^-1*w2*S2)^2, (w2*

6 17 96 [w2^2, (w2*S2^-1)^3, S2^8, w2*S2^-1*(S2^-2*w2)^2*S2*w2*S2^-2]
6 19 96 [w2^2, (w2*S2^-1)^3, S2^8, w2*S2^-1*(S2^-2*w2)^2*S2*w2*S2^-2]
7 1 128 [w2^2, (w2*S2^-1)^4, S2^8, w2*S2*(S2^2*w2)^2*S2*w2*S2^-2]
7 5 128 [w2^2, (w2*S2^-1)^4, S2^8, w2*S2*(S2^2*w2)^2*S2*w2*S2^-2]
7 7 128 [w2^2, (w2*S2^-1)^4, S2^8, w2*S2*(S2^2*w2)^2*S2*w2*S2^-2]
7 11 128 [w2^2, (w2*S2^-1)^4, S2^8, w2*S2*(S2^2*w2)^2*S2*w2*S2^-2]
7 13 128 [w2^2, (w2*S2^-1)^4, S2^8, w2*S2*(S2^2*w2)^2*S2*w2*S2^-2]
7 17 128 [w2^2, (w2*S2^-1)^4, S2^8, w2*S2*(S2^2*w2)^2*S2*w2*S2^-2]
7 19 128 [w2^2, (w2*S2^-1)^4, S2^8, w2*S2*(S2^2*w2)^2*S2*w2*S2^-2]
8 1 128 [w2^2, S2^8, (w2*S2)^2*S2*(w2*S2^-1)^2*S2^-1, (w2*S2^-3)^2*w2*S2^-1*w2*S2^-1]
8 5 128 [w2^2, S2^8, (w2*S2)^2*S2*(w2*S2^-1)^2*S2^-1, (w2*S2^-3)^2*w2*S2^-1*w2*S2^-1]
8 7 128 [w2^2, S2^8, (w2*S2)^2*S2*(w2*S2^-1)^2*S2^-1, (w2*S2^-3)^2*w2*S2^-1*w2*S2^-1]
8 11 128 [w2^2, S2^8, (w2*S2)^2*S2*(w2*S2^-1)^2*S2^-1, (w2*S2^-3)^2*w2*S2^-1*w2*S2^-1]


8 13 128 [w2^2, S2^8, (w2*S2)^2*S2*(w2*S2^-1)^2*S2^-1, (w2*S2^-3)^2*w2*S2^-1*w2*S2^-1]
8 17 128 [w2^2, S2^8, (w2*S2)^2*S2*(w2*S2^-1)^2*S2^-1, (w2*S2^-3)^2*w2*S2^-1*w2*S2^-1]
8 19 128 [w2^2, S2^8, (w2*S2)^2*S2*(w2*S2^-1)^2*S2^-1, (w2*S2^-3)^2*w2*S2^-1*w2*S2^-1]
9 1 128 [w2^2, w2*S2*(w2*S2^-1)^2*w2*S2, S2^8]
9 5 128 [w2^2, w2*S2*(w2*S2^-1)^2*w2*S2, S2^8]
9 7 128 [w2^2, w2*S2*(w2*S2^-1)^2*w2*S2, S2^8]


9 11 128 [w2^2, w2*S2*(w2*S2^-1)^2*w2*S2, S2^8]
9 13 128 [w2^2, w2*S2*(w2*S2^-1)^2*w2*S2, S2^8]
9 17 128 [w2^2, w2*S2*(w2*S2^-1)^2*w2*S2, S2^8]


9 19 128 [w2^2, w2*S2*(w2*S2^-1)^2*w2*S2, S2^8]
10 1 128 [w2^2, w2*S2*(w2*S2^-1)^2*w2*S2, S2^8]
10 5 128 [w2^2, w2*S2*(w2*S2^-1)^2*w2*S2, S2^8]
10 7 128 [w2^2, w2*S2*(w2*S2^-1)^2*w2*S2, S2^8]


...

11 7 128 [w2^2, w2*S2*(w2*S2^-1)^2*w2*S2, S2^8]


11 11 128 [w2^2, w2*S2*(w2*S2^-1)^2*w2*S2, S2^8]


11 13 128 [w2^2, w2*S2*(w2*S2^-1)^2*w2*S2, S2^8]


11 17 128 [w2^2, w2*S2*(w2*S2^-1)^2*w2*S2, S2^8]


11 19 128 [w2^2, w2*S2*(w2*S2^-1)^2*w2*S2, S2^8]


In [20]:
for i in range(1,7):
    B0,invariants = B0_as_finitely_presented_group(3**i)
    for N in range(1,20,2):
        if N%3==0:
            continue
        E0, names = E0_as_permutation_group(3**i*N, full_level=True)
        print(i, N, E0.cardinality(),E0_relations_on_cusps(3**i*N, full_level=True))
        
        phi = B0.hom(E0.gens(), E0)
        # sage doesn't have is_isomorphism for phi
        # so we do some cadrinality checks which are equivalent
        assert phi.kernel().cardinality() == 1
        assert B0.cardinality() == E0.cardinality()

1 1 2 [S3, w3^2]
1 5 2 [S3, w3^2]
1 7 2 [S3, w3^2]
1 11 2 [S3, w3^2]
1 13 2 [S3, w3^2]
1 17 2 [S3, w3^2]
1 19 2 [S3, w3^2]
2 1 12 [w3^2, S3^3, (w3*S3^-1)^3]
2 5 12 [w3^2, S3^3, (w3*S3^-1)^3]
2 7 12 [w3^2, S3^3, (w3*S3^-1)^3]
2 11 12 [w3^2, S3^3, (w3*S3^-1)^3]
2 13 12 [w3^2, S3^3, (w3*S3^-1)^3]
2 17 12 [w3^2, S3^3, (w3*S3^-1)^3]
2 19 12 [w3^2, S3^3, (w3*S3^-1)^3]
3 1 18 [w3^2, S3^3, w3*S3*(w3*S3^-1)^2*w3*S3]
3 5 18 [w3^2, S3^3, w3*S3*(w3*S3^-1)^2*w3*S3]
3 7 18 [w3^2, S3^3, w3*S3*(w3*S3^-1)^2*w3*S3]
3 11 18 [w3^2, S3^3, w3*S3*(w3*S3^-1)^2*w3*S3]
3 13 18 [w3^2, S3^3, w3*S3*(w3*S3^-1)^2*w3*S3]
3 17 18 [w3^2, S3^3, w3*S3*(w3*S3^-1)^2*w3*S3]
3 19 18 [w3^2, S3^3, w3*S3*(w3*S3^-1)^2*w3*S3]
4 1 18 [w3^2, S3^3, w3*S3*(w3*S3^-1)^2*w3*S3]
4 5 18 [w3^2, S3^3, w3*S3*(w3*S3^-1)^2*w3*S3]
4 7 18 [w3^2, S3^3, w3*S3*(w3*S3^-1)^2*w3*S3]
4 11 18 [w3^2, S3^3, w3*S3*(w3*S3^-1)^2*w3*S3]
4 13 18 [w3^2, S3^3, w3*S3*(w3*S3^-1)^2*w3*S3]
4 17 18 [w3^2, S3^3, w3*S3*(w3*S3^-1)^2*w3*S3]
4 19 18 [w3^2, S3^3, w3*S3*(w3

5 7 18 [w3^2, S3^3, w3*S3*(w3*S3^-1)^2*w3*S3]
5 11 18 [w3^2, S3^3, w3*S3*(w3*S3^-1)^2*w3*S3]
5 13 18 [w3^2, S3^3, w3*S3*(w3*S3^-1)^2*w3*S3]
5 17 18 [w3^2, S3^3, w3*S3*(w3*S3^-1)^2*w3*S3]
5 19 18 [w3^2, S3^3, w3*S3*(w3*S3^-1)^2*w3*S3]
6 1 18 [w3^2, S3^3, w3*S3*(w3*S3^-1)^2*w3*S3]
6 5 18 [w3^2, S3^3, w3*S3*(w3*S3^-1)^2*w3*S3]


6 7 18 [w3^2, S3^3, w3*S3*(w3*S3^-1)^2*w3*S3]
6 11 18 [w3^2, S3^3, w3*S3*(w3*S3^-1)^2*w3*S3]


6 13 18 [w3^2, S3^3, w3*S3*(w3*S3^-1)^2*w3*S3]
6 17 18 [w3^2, S3^3, w3*S3*(w3*S3^-1)^2*w3*S3]


6 19 18 [w3^2, S3^3, w3*S3*(w3*S3^-1)^2*w3*S3]


In [23]:
for i in range(1,12):
    for j in range(1,6):
        B0,invariants = B0_as_finitely_presented_group(2**i*3**j)
        E0, names = E0_as_permutation_group(2**i*3**j)
        print(i, j, E0.cardinality(),E0_relations_on_cusps(2**i*3**j))
        
        phi = B0.hom(E0.gens(), E0)
        # sage doesn't have is_isomorphism for phi
        # so we do some cadrinality checks which are equivalent
        assert phi.kernel().cardinality() == 1
        assert B0.cardinality() == E0.cardinality()

1 1 4 [S3, S2, w2^2, w3^2, (w2*w3)^2]
1 2 24 [S2, w2^2, w3^2, S3^3, (w2*w3)^2, (w2*S3^-1)^2, (w3*S3^-1)^3]
1 3 36 [S2, w2^2, w3^2, S3^3, (w2*w3)^2, (w2*S3^-1)^2, w3*S3*(w3*S3^-1)^2*w3*S3]
1 4 36 [S2, w2^2, w3^2, S3^3, (w2*w3)^2, (w2*S3^-1)^2, w3*S3*(w3*S3^-1)^2*w3*S3]
1 5 36 [S2, w2^2, w3^2, S3^3, (w2*w3)^2, (w2*S3^-1)^2, w3*S3*(w3*S3^-1)^2*w3*S3]
2 1 12 [S3, S2^2, w2^2, w3^2, (w3*S2)^2, (w2*w3)^2, (w2*S2)^3]
2 2 72 [w2^2, S2^2, w3^2, S3^3, (S2*w3)^2, (w3*w2)^2, S2*S3^-1*S2*S3, w2*S3*w2*S3^-1, (w3*S3^-1)^3, (S2*w2)^3]
2 3 108 [S2^2, w2^2, w3^2, S3^3, (w3*S2)^2, (w2*w3)^2, w2*S3^-1*w2*S3, S2*S3*S2*S3^-1, (w2*S2)^3, w3*S3*(w3*S3^-1)^2*w3*S3]
2 4 108 [S2^2, w2^2, w3^2, S3^3, (w3*S2)^2, (w2*w3)^2, w2*S3^-1*w2*S3, S2*S3*S2*S3^-1, (w2*S2)^3, w3*S3*(w3*S3^-1)^2*w3*S3]
2 5 108 [S2^2, w2^2, w3^2, S3^3, (w3*S2)^2, (w2*w3)^2, w2*S3^-1*w2*S3, S2*S3*S2*S3^-1, (w2*S2)^3, w3*S3*(w3*S3^-1)^2*w3*S3]
3 1 16 [S3, S2^2, w2^2, w3^2, (w3*S2)^2, (w2*w3)^2, (w2*S2)^4]
3 2 96 [S2^2, w2^2, w3^2, S3^3, (w3*S2)^2

3 5 144 [S2^2, w2^2, w3^2, S3^3, (w3*S2)^2, (w2*w3)^2, (w2*S3^-1)^2, S2*S3*S2*S3^-1, w3*S3*(w3*S3^-1)^2*w3*S3, (w2*S2)^4]
4 1 48 [S3, w2^2, w3^2, (w2*w3)^2, S2^4, (w3*S2^-1)^2, (w2*S2^-1)^3]
4 2 288 [w2^2, w3^2, S3^3, w3*S2^-1*w3*S2, (w2*w3)^2, w2*S3^-1*w2*S3, S2^4, S2*S3^-1*S2^-1*S3, (w3*S3^-1)^3, (w2*S2^-1)^3]
4 3 432 [w2^2, w3^2, S3^3, (w3*S2^-1)^2, (w2*w3)^2, w2*S3^-1*w2*S3, S2^4, S2*S3^-1*S2^-1*S3, (w2*S2^-1)^3, w3*S3*(w3*S3^-1)^2*w3*S3]
4 4 432 [w2^2, w3^2, S3^3, w3*S2^-1*w3*S2, (w2*w3)^2, w2*S3^-1*w2*S3, S2^4, S2*S3^-1*S2^-1*S3, (w2*S2^-1)^3, w3*S3*(w3*S3^-1)^2*w3*S3]


4 5 432 [w2^2, w3^2, S3^3, (w3*S2^-1)^2, (w2*w3)^2, w2*S3^-1*w2*S3, S2^4, S2*S3^-1*S2^-1*S3, (w2*S2^-1)^3, w3*S3*(w3*S3^-1)^2*w3*S3]
5 1 64 [S3, w2^2, w3^2, (w3*S2^-1)^2, (w2*w3)^2, S2^4, (w2*S2^-1*w2*S2)^2, (w2*S2^-1)^4]
5 2 384 [w2^2, w3^2, S3^3, w3*S2^-1*w3*S2, (w2*w3)^2, (w2*S3^-1)^2, S2^4, S2*S3^-1*S2^-1*S3, (w3*S3^-1)^3, (w2*S2^-1*w2*S2)^2, (w2*S2^-1)^4]
5 3 576 [w2^2, w3^2, S3^3, (w3*S2^-1)^2, (w2*w3)^2, (w2*S3^-1)^2, S2^4, S2*S3^-1*S2^-1*S3, w3*S3*(w3*S3^-1)^2*w3*S3, (w2*S2^-1*w2*S2)^2, (w2*S2^-1)^4]
5 4 576 [w2^2, w3^2, S3^3, (w2*w3)^2, (w2*S3^-1)^2, S2^4, w3*S2*w3*S2^-1, S3*S2^-1*S3^-1*S2, w3*S3*(w3*S3^-1)^2*w3*S3, (w2*S2^-1*w2*S2)^2, (w2*S2^-1)^4]


5 5 576 [w2^2, w3^2, S3^3, (w3*S2^-1)^2, (w2*w3)^2, (w2*S3^-1)^2, S2^4, S2*S3^-1*S2^-1*S3, w3*S3*(w3*S3^-1)^2*w3*S3, (w2*S2^-1*w2*S2)^2, (w2*S2^-1)^4]
6 1 192 [S3, w2^2, w3^2, (w2*w3)^2, (S2^2*w3)^2, (w2*S2^-1)^3, S2^-2*w3*S2*S3*w3*S2^-1, (S3*(S2^-1*w3)^2)^2]
6 2 1152 [w2^2, w3^2, S3^3, w3*S2^-1*w3*S2, (w2*w3)^2, w2*S3^-1*w2*S3, S2*S3^-1*S2^-1*S3, (w3*S3^-1)^3, (w2*S2^-1)^3, S2^8, w2*S2^-1*(S2^-2*w2)^2*S2*w2*S2^-2]
6 3 1728 [w2^2, w3^2, S3^3, (w2*w3)^2, w2*S3^-1*w2*S3, S2*S3^-1*S2^-1*S3, S2^2*w3*S2^-1*w3*S2, (w2*S2^-1)^3, w3*S3*(w3*S3^-1)^2*w3*S3]


6 4 1728 [w2^2, w3^2, S3^3, w3*S2^-1*w3*S2, (w2*w3)^2, w2*S3^-1*w2*S3, S2*S3^-1*S2^-1*S3, (w2*S2^-1)^3, w3*S3*(w3*S3^-1)^2*w3*S3, S2^8, w2*S2^-1*(S2^-2*w2)^2*S2*w2*S2^-2]


...

11 1 256 [S3, w2^2, w3^2, (w2*w3)^2, S2^-1*w3*S2*w3*S2^-2, w2*S2*(w2*S2^-1)^2*w2*S2]


11 2 1536 [w2^2, w3^2, S3^3, w3*S2^-1*w3*S2, (w2*w3)^2, (w2*S3^-1)^2, S2*S3^-1*S2^-1*S3, (w3*S3^-1)^3, w2*S2*(w2*S2^-1)^2*w2*S2, S2^8]


11 3 2304 [w2^2, w3^2, S3^3, (w2*w3)^2, (w2*S3^-1)^2, S2*S3^-1*S2^-1*S3, S2^2*w3*S2^-1*w3*S2, w3*S3*(w3*S3^-1)^2*w3*S3, w2*S2*(w2*S2^-1)^2*w2*S2]


11 4 2304 [w2^2, w3^2, S3^3, w3*S2^-1*w3*S2, (w2*w3)^2, (w2*S3^-1)^2, S2*S3^-1*S2^-1*S3, w3*S3*(w3*S3^-1)^2*w3*S3, w2*S2*(w2*S2^-1)^2*w2*S2, S2^8]


11 5 2304 [w2^2, w3^2, S3^3, (w2*w3)^2, (w2*S3^-1)^2, S2*S3^-1*S2^-1*S3, S2^2*w3*S2^-1*w3*S2, w3*S3*(w3*S3^-1)^2*w3*S3, w2*S2*(w2*S2^-1)^2*w2*S2]


In [0]:
1+1

In [55]:
E0 = E0_as_matrix_group(2**5)[0]

In [72]:
for N in srange(2,1000):
    G = Gamma0(N)
    if G.genus()==0:
        continue
        
    M = N.prime_to_m_part(6)
    if M == N:
        # in this case E0 is trivial
        continue
    
    B0, invariants = B0_as_finitely_presented_group(N//M)
    E0, names = E0_as_matrix_group(N)
    print(N, E0.cardinality(), B0.cardinality())
    
    if E0.cardinality() != B0.cardinality():
        print(f"Not an isomomorphism for N={N}, g(X0(N))={G.genus()}, #B0({N//M})={B0.cardinality()} #E0({N}){E0.cardinality()}")
        
        


14 1 2
Not an isomomorphism for N=14, g(X0(N))=1, #B0(2)=2 #E0(14)1
15 1 2
Not an isomomorphism for N=15, g(X0(N))=1, #B0(3)=2 #E0(15)1
20 2 6
Not an isomomorphism for N=20, g(X0(N))=1, #B0(4)=6 #E0(20)2
21 2 2
22 2 2
24 2 16
Not an isomomorphism for N=24, g(X0(N))=1, #B0(24)=16 #E0(24)2
26 2 2
27 6 18
Not an isomomorphism for N=27, g(X0(N))=1, #B0(27)=18 #E0(27)6
28 6 6
30 4 4
32 4 32
Not an isomomorphism for N=32, g(X0(N))=1, #B0(32)=32 #E0(32)4
33 2 2
34 2 2
36 6 72
Not an isomomorphism for N=36, g(X0(N))=1, #B0(36)=72 #E0(36)6
38 2 2
39 2 2
40 8 8
42 4 4
44 6 6
45 12 12
46 2 2
48 48 48
50 2 2
51 2 2
52 6 6
54 36 36
56 8 8
57 2 2
58 2 2
60 12 12
62 2 2
63 12 12


64 96 96
66 4 4
68 6 6
69 2 2
70 2 2
72 96 96
74 2 2
75 2 2
76 6 6
78 4 4
80 24 24
81 18 18
82 2 2


84 12 12
86 2 2
87 2 2
88 8 8
90 24 24
92 6 6
93 2 2
94 2 2
96 64 64
98 2 2


99 12 12
100 6 6
102 4 4
104 8 8


105 2 2
106 2 2
108 108 108
110 2 2


...

993 2 2


994 2 2


996 12 12


998 2 2


999 18 18


In [116]:
for i in range(0,12):
    for j in range(0,6):
        if [i,j]==[0,0]:
            continue
        E0, names = E0_as_permutation_group(2**i*3**j)
        nr_E0 = E0.cardinality()
        print(i, j, E0.is_solvable(), E0.group_id() if nr_E0 < 1536 else [nr_E0, "?"] , E0.structure_description())

0 1 True [2, 1] C2
0 2 True [12, 3] A4
0 3 True [18, 3] C3 x S3
0 4 True [18, 3] C3 x S3
0 5 True [18, 3] C3 x S3
1 0 True [2, 1] C2
1 1 True [4, 2] C2 x C2
1 2 True [24, 12] S4
1 3 True [36, 10] S3 x S3
1 4 True [36, 10] S3 x S3
1 5 True [36, 10] S3 x S3
2 0 True [6, 1] S3
2 1 True [12, 4] D6
2 2 True [72, 44] A4 x S3
2 3 True [108, 38] C3 x S3 x S3
2 4 True [108, 38] C3 x S3 x S3
2 5 True [108, 38] C3 x S3 x S3
3 0 True [8, 3] D4
3 1 True [16, 11] C2 x D4
3 2 True [96, 195] (C2 x S4) : C2


3 3 True [144, 153] ((C6 x C2) : C2) x S3
3 4 True [144, 153] ((C6 x C2) : C2) x S3
3 5 True [144, 153] ((C6 x C2) : C2) x S3
4 0 True [24, 12] S4
4 1 True [48, 48] C2 x S4
4 2 True [288, 1024] S4 x A4
4 3 True [432, 745] C3 x S4 x S3


4 4 True [432, 745] C3 x S4 x S3
4 5 True [432, 745] C3 x S4 x S3
5 0 True [32, 6] (C2 x C2 x C2) : C4
5 1 True [64, 138] ((C2 x C2 x C2) : (C2 x C2)) : C2


5 2 True [384, 5625] (((C4 x C2) : C2) x A4) : C2
5 3 True [576, 2289] (C3 x C3) : (((C2 x C2 x C2) : (C2 x C2)) : C2)


5 4 True [576, 2285] (C3 : ((C2 x C2 x C2) : C4)) x S3


...

11 1 True [256, 5352] ((C8 . D4 = C4 . (C8 x C2)) : C2) : C2


11 2 True [1536, '?'] (C8 x C8 x A4) : C2


11 3 True [2304, '?'] (C3 x C3) : (((C8 . D4 = C4 . (C8 x C2)) : C2) : C2)


11 4 True [2304, '?'] (C3 : ((C8 x C8) : C2)) x S3


11 5 True [2304, '?'] (C3 x C3) : (((C8 . D4 = C4 . (C8 x C2)) : C2) : C2)


In [10]:
for i in range(1,12):
    E0, names = E0_as_permutation_group(2**i)
    E0gap = E0.gap()
    E0pcgs = E0gap.Pcgs()
    print([E0gap.Factorization(g) for g in E0pcgs])

[x1]
[x1*x2*x1, x2*x1]
[x1*x2*x1, x2*x1*x2, x1]
[x2, x1*x2^-1*x1*x2, x2^2, x1*x2^2*x1]
[x1*x2*x1, x1*x2^2*x1, x2^2*x1*x2^2, x2^-1*x1*x2, x1]
[x2, x1*x2^-1*x1*x2, x2^2, x1*x2^-2*x1, x2^4, x1*x2^4*x1]
[x1*x2*x1, x1*x2^2*x1, x1*x2^4*x1, x2^-2*x1*x2^2, x2^-1*x1*x2, x1, (x2^-1*x1*x2*x1)^2]
[x1*x2*x1, x1*x2^2*x1, x1*x2^4*x1, x2^-1*x1*x2, x1, x2^2*x1*x2^-2*x1, (x2^-1*x1)^2*(x2*x1)^2]
[x1*x2*x1, x1*x2^2*x1, x1*x2^4*x1, x2^-1*x1*x2, x1, x2^-2*x1*x2^2*x1, (x2^4*x1)^2]
[x1*x2*x1, x1*x2^2*x1, x1*x2^4*x1, x2^-1*x1*x2, x1, x2^-2*x1*x2^2*x1, (x2^4*x1)^2]
[x1*x2*x1, x1*x2^2*x1, x1*x2^4*x1, x2^-1*x1*x2, x1, x2^-2*x1*x2^2*x1, (x2^4*x1)^2]


In [117]:
for i in range(1,11):
    E0, names = E0_as_permutation_group(2**i)
    print(i,E0.group_id(),E0.structure_description())

1 [2, 1] C2
2 [6, 1] S3
3 [8, 3] D4
4 [24, 12] S4
5 [32, 6] (C2 x C2 x C2) : C4
6 [96, 64] ((C4 x C4) : C3) : C2
7 [128, 136] ((C8 : C4) : C2) : C2


8 [128, 68] (C8 : C8) : C2
9 [128, 67] (C8 x C8) : C2
10 [128, 67] (C8 x C8) : C2


In [118]:
for i in range(1,5):
    E0, names = E0_as_permutation_group(3**i)
    print(i,E0.group_id(),E0.structure_description())

1 [2, 1] C2
2 [12, 3] A4
3 [18, 3] C3 x S3
4 [18, 3] C3 x S3


In [20]:
for i in range(1,13):
    for j in range(1,7):
        if [i,j]==[0,0]:
            continue
        E0, names = E0_as_permutation_group(2**i*3**j)
        nr_E0 = E0.cardinality()
        assert E0.is_solvable()
        E0_2 = E0.subgroup([E0.gen(0),E0.gen(1)])
        E0_3 = E0.subgroup([E0.gen(2),E0.gen(3)])
        assert E0.cardinality() == E0_2.cardinality()*E0_3.cardinality()
        assert E0_2.intersection(E0_3).cardinality()==1
        print(i, j, E0_2.is_normal(E0),E0_3.is_normal(E0), E0.group_id() if nr_E0 < 1536 else [nr_E0, "?"] , E0.structure_description())

1 1 True True [4, 2] C2 x C2
1 2 False True [24, 12] S4
1 3 False True [36, 10] S3 x S3
1 4 False True [36, 10] S3 x S3
1 5 False True [36, 10] S3 x S3
1 6 False True [36, 10] S3 x S3
2 1 True True [12, 4] D6
2 2 True True [72, 44] A4 x S3
2 3 True True [108, 38] C3 x S3 x S3
2 4 True True [108, 38] C3 x S3 x S3
2 5 True True [108, 38] C3 x S3 x S3


2 6 True True [108, 38] C3 x S3 x S3
3 1 True True [16, 11] C2 x D4
3 2 False True [96, 195] (C2 x S4) : C2
3 3 False True [144, 153] ((C6 x C2) : C2) x S3
3 4 False True [144, 153] ((C6 x C2) : C2) x S3
3 5 False True [144, 153] ((C6 x C2) : C2) x S3


3 6 False True [144, 153] ((C6 x C2) : C2) x S3
4 1 True False [48, 48] C2 x S4
4 2 True True [288, 1024] S4 x A4
4 3 True False [432, 745] C3 x S4 x S3
4 4 True True [432, 745] C3 x S4 x S3


4 5 True False [432, 745] C3 x S4 x S3


4 6 True True [432, 745] C3 x S4 x S3
5 1 True False [64, 138] ((C2 x C2 x C2) : (C2 x C2)) : C2
5 2 False True [384, 5625] (((C4 x C2) : C2) x A4) : C2


...

12 2 True True [1536, '?'] ((C8 x C8) : C2) x A4


12 3 True False [2304, '?'] C3 x (C3 : (((C8 : Q8) : C2) : C2))


12 4 True True [2304, '?'] C3 x (((C8 x C8) : C2) x S3)


12 5 True False [2304, '?'] C3 x (C3 : (((C8 : Q8) : C2) : C2))


12 6 True True [2304, '?'] C3 x (((C8 x C8) : C2) x S3)


In [13]:
E0, names = E0_as_permutation_group(2**3*3**3)
E0_2 = E0.subgroup([E0.gen(0),E0.gen(1)])
E0_3 = E0.subgroup([E0.gen(2),E0.gen(3)])

True

In [1]:
NoCM = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 12, 13, 15, 16, 17, 18, 20, 21, 22, 23, 24, 25, 26, 28, 29, 30, 31, 32, 33, 35, 36, 37, 39, 40, 41, 46, 47, 48, 49, 50, 53, 59, 61, 65, 71, 79, 83, 89, 101, 131]
print([N for N in NoCM if Gamma0(N).genus()>2])
#[30, 33, 35, 39, 40, 41, 46, 47, 48, 53, 59, 61, 65, 71, 79, 83, 89, 101, 131]

# X_0(N) known to have infintely many degree 2 points
#1-33,35-37,39-41,43,46-50,53,59,61,65,71,79,83,89,101,131

[30, 33, 35, 39, 40, 41, 46, 47, 48, 53, 59, 61, 65, 71, 79, 83, 89, 101, 131]


In [11]:
1+1

2